In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from utils import generate_with_single_input, generate_with_multiple_input

In [3]:
# Example: single input
output = generate_with_single_input(
    prompt="What is the capital of France?"
)

print("Role:", output['role'])
print("Content:", output['content'])

Role: assistant
Content: The capital of France is Paris.


In [4]:
# Example: multiple inputs (conversation)
messages = [
    {'role': 'user', 'content': 'Hello, who won the FIFA world cup in 2018?'},
    {'role': 'assistant', 'content': 'France won the 2018 FIFA World Cup.'},
    {'role': 'user', 'content': 'Who was the captain?'}
]

output = generate_with_multiple_input(
    messages=messages,
    max_tokens=100
)

print("Role:", output['role'])
print("Content:", output['content'])

Role: assistant
Content: The captain of the French national team during the 2018 FIFA World Cup was Hugo Lloris.


<a id='2'></a>
## 2 - Integrating Data into an LLM Prompt

In this section, you will learn how to effectively incorporate data into a prompt before passing it to a Large Language Model (LLM). We will work with a small dataset consisting of JSON files that contain information about houses. It will help you understand how to augment prompts in the context of RAG.

<a id='2-1'></a>
### 2.1 Understanding the data structure

Let's have a quick look in the data structure. It is a tiny dataset of houses. A list containing one dictionary per house.

In [5]:
house_data = [
    {
        "address": "123 Maple Street",
        "city": "Springfield",
        "state": "IL",
        "zip": "62701",
        "bedrooms": 3,
        "bathrooms": 2,
        "square_feet": 1500,
        "price": 230000,
        "year_built": 1998
    },
    {
        "address": "456 Elm Avenue",
        "city": "Shelbyville",
        "state": "TN",
        "zip": "37160",
        "bedrooms": 4,
        "bathrooms": 3,
        "square_feet": 2500,
        "price": 320000,
        "year_built": 2005
    }
]

<a id='2-2'></a>
### 2.2 Creating the Prompt

Let's begin by constructing the prompt. The first step is to design a layout for the data.

In [6]:
# First, let's create a layout for the houses

def house_info_layout(houses):
    # Create an empty string
    layout = ''
    # Iterate over the houses
    for house in houses:
        # For each house, append the information to the string using f-strings
        # The following way using brackets is a good way to make the code readable as in each line you can start a new f-string that will appended to the previous one
        layout += (f"House located at {house['address']}, {house['city']}, {house['state']} {house['zip']} with "
            f"{house['bedrooms']} bedrooms, {house['bathrooms']} bathrooms, "
            f"{house['square_feet']} sq ft area, priced at ${house['price']}, "
            f"built in {house['year_built']}.\n") # Don't forget the new line character at the end!
    return layout

In [7]:
# Check the layout
print(house_info_layout(house_data))

House located at 123 Maple Street, Springfield, IL 62701 with 3 bedrooms, 2 bathrooms, 1500 sq ft area, priced at $230000, built in 1998.
House located at 456 Elm Avenue, Shelbyville, TN 37160 with 4 bedrooms, 3 bathrooms, 2500 sq ft area, priced at $320000, built in 2005.



Now create a function that generates the prompt to be passed to the Language Learning Model (LLM). The function will take a user-provided query and the available housing data as inputs to effectively address the user's query.

In [8]:
def generate_prompt(query, houses):
    # The code made above is modular enough to accept any list of houses, so you could also choose a subset of the dataset.
    # This might be useful in a more complex context where you want to give only some information to the LLM and not the entire data
    houses_layout = house_info_layout(houses)
    # Create a hard-coded prompt. You can use three double quotes (") in this cases, so you don't need to worry too much about using single or double quotes and breaking the code
    PROMPT = f"""
Use the following houses information to answer users queries.
{houses_layout}
Query: {query}    
             """
    return PROMPT

In [9]:
print(generate_prompt("What is the most expensive house?", houses = house_data))


Use the following houses information to answer users queries.
House located at 123 Maple Street, Springfield, IL 62701 with 3 bedrooms, 2 bathrooms, 1500 sq ft area, priced at $230000, built in 1998.
House located at 456 Elm Avenue, Shelbyville, TN 37160 with 4 bedrooms, 3 bathrooms, 2500 sq ft area, priced at $320000, built in 2005.

Query: What is the most expensive house?    
             


Now let's call the LLM!

In [10]:
query = "What is the most expensive house? And the bigger one?"
# Asking without the augmented prompt, let's pass the role as user
query_without_house_info = generate_with_single_input(prompt = query, role = 'user')
# With house info, given the prompt structuer, let's pass the role as assistant
enhanced_query = generate_prompt(query, houses = house_data)
query_with_house_info = generate_with_single_input(prompt = enhanced_query, role = 'assistant')

In [11]:
# Without house info
print(query_without_house_info['content'])

As of my last knowledge update in October 2021, the most expensive house in the world was believed to be "Antilia," a private residence in Mumbai, India, owned by billionaire Mukesh Ambani. Antilia is a 27-story skyscraper that reportedly cost over $2 billion to build and features luxurious amenities such as multiple swimming pools, a health spa, a ballroom, and a garage that can accommodate 168 cars.

In terms of size, the largest private residence is generally recognized as "Biltmore Estate" in Asheville, North Carolina, which spans about 178,926 square feet. However, there are larger homes, such as the "Palace of the Parliament" in Bucharest, Romania, but it is not a private residence.

For the most accurate information, especially regarding current records of expensive or large houses, I recommend checking recent real estate listings or news updates, as these records can change frequently.


In [12]:
# With house info
print(query_with_house_info['content'])

The most expensive house is located at 456 Elm Avenue, Shelbyville, TN 37160, priced at $320,000. The bigger house is also located at 456 Elm Avenue, with an area of 2,500 sq ft.
